In [94]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import time
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing, decomposition, model_selection, metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from bs4 import BeautifulSoup


# from keras.preprocessing import sequence, text
# from keras.models import Sequential

In [95]:
from sklearn import pipeline

In [96]:
import jieba
#处理训练集，将训练集的文本信息和label信息合并，清洗特殊符号，同时将文本内容进行分词
def merge_feature_label(feature_name,label_name):
    feature=pd.read_csv(feature_name,sep=",")
    label=pd.read_csv(label_name,sep=",")
    data=feature.merge(label,on='id')
    data["X"]=data[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
    # 丢掉缺失值
    dataDropNa=data.dropna(axis=0, how='any')
    print(dataDropNa.info())
    dataDropNa["X"]=dataDropNa["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
    dataDropNa["X_split"]=dataDropNa["X"].apply(lambda x:" ".join(jieba.cut(x)))
    return dataDropNa
 
dataDropNa=merge_feature_label("Train_DataSet.csv","Train_DataSet_Label.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7265 entries, 0 to 7339
Data columns (total 5 columns):
id         7265 non-null object
title      7265 non-null object
content    7265 non-null object
label      7265 non-null int64
X          7265 non-null object
dtypes: int64(1), object(4)
memory usage: 340.5+ KB
None


C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [102]:
dataDropNa.label.unique()

array([2, 1, 0], dtype=int64)

In [103]:
data = dataDropNa

In [121]:
type(data)

pandas.core.frame.DataFrame

In [104]:
type(data["X_split"])

pandas.core.series.Series

In [105]:
# data["X_split"].head

In [106]:
#处理测试数据
def process_test(test_name):
    test=pd.read_csv(test_name,sep=",")
    # apply()方法能劫持另外一个对象的方法，继承另外一个对象的属性 
    # lambda https://blog.csdn.net/zjuxsl/article/details/79437563
    #  join() 方法用于将序列中的元素以指定的字符连接生成一个新的字符串。
    test["X"]=test[["title","content"]].apply(lambda x:"".join([str(x[0]),str(x[1])]),axis=1)
    # 列名信息
    print(test.info())
    # replace(old, new) 方法把字符串中的 old（旧字符串） 替换成 new(新字符串)，如果指定第三个参数max，则替换不超过 max 次。
    test["X"]=test["X"].apply(lambda x: str(x).replace("\\n","").replace(".","").replace("\n","").replace("　","").replace("↓","").replace("/","").replace("|","").replace(" ",""))
    # jieba.cut()按照词性切分“我爱中国”切分成我爱中国
    test["X_split"]=test["X"].apply(lambda x:" ".join(jieba.cut(x)))
    return test
 
testData=process_test("Test_DataSet.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7356 entries, 0 to 7355
Data columns (total 4 columns):
id         7356 non-null object
title      7356 non-null object
content    7288 non-null object
X          7356 non-null object
dtypes: object(4)
memory usage: 230.0+ KB
None


In [113]:
test_data = testData


In [ ]:
# data.head()

In [8]:
def multiclass_logloss(actual, predicted, eps = 1e-15):
    "对数损失度量"
    
    if len(actual.shape)==1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2
        
    clip = np.clip(predicted, eps, 1-eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota
    

In [125]:
# lbl_enc = preprocessing.LabelEncoder()
# y = lbl_enc.fit_transform(data.label.values)
xdata = data.X_split.values # 总训练集 文本
y = data.label.values # 训练集label
ydata = test_data.X_split.values #测试集 文本

In [92]:
xtrain, xvalid, ytrain, yvalid = train_test_split(xdata, y,
                                                  stratify=y, 
                                                  random_state=42,
                                                  test_size=0.1, shuffle=True)

In [69]:
print (xtrain.shape) 
print (xvalid.shape)

(6538,)
(727,)


In [137]:
ydata.shape

(7356,)

In [131]:
type(ytrain)

numpy.ndarray

In [127]:
type(xdata)

numpy.ndarray

In [126]:
type(ydata)

numpy.ndarray

**构建基础模型(Basic Models)**

简单的 TF-IDF (Term Frequency - Inverse Document Frequency)+逻辑斯底回归(Logistic Regression).

In [12]:
def number_normalizer(tokens): 
    """ 将所有数字标记映射为一个占位符(Placeholder)。 
    对于许多实际应用场景来说,以数字开头的tokens不是很有用, 但这样tokens的存在也有一定相关性。 
    通过将所有数字都表示成同一个符号,可以达到降维的目的。 
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)
class NumberNormalizingVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super(NumberNormalizingVectorizer, self).build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc)))

利用刚才创建的NumberNormalizingVectorizer类来提取文本特征

In [52]:
tfv = NumberNormalizingVectorizer(min_df=3, 
                                  max_df=0.5,
                                  max_features=None, 
                                  ngram_range=(1, 2), 
                                  use_idf=True, 
                                  smooth_idf=True)

In [115]:
print(tfv)

NumberNormalizingVectorizer(analyzer='word', binary=False,
              decode_error='strict', dtype=<class 'numpy.int64'>,
              encoding='utf-8', input='content', lowercase=True,
              max_df=0.5, max_features=None, min_df=3, ngram_range=(1, 2),
              norm='l2', preprocessor=None, smooth_idf=True,
              stop_words=None, strip_accents=None, sublinear_tf=False,
              token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None,
              use_idf=True, vocabulary=None)


使用TF-IDF来fit训练集和测试集(半监督学习)

In [14]:
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv = tfv.transform(xtrain)
xvalid_tfv = tfv.transform(xvalid)




利用提取的TFIDF特征来fit一个简单的Logistic Regression :

In [15]:
clf = LogisticRegression(C=1.0,solver='lbfgs',multi_class='multinomial')
clf.fit(xtrain_tfv, ytrain) 
predictions = clf.predict_proba(xvalid_tfv)
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.642 


使用词汇计数(Word Counts)作为功能,而不是使用TF-IDF。 这可以使用scikit-learn

In [16]:
ctv = CountVectorizer(min_df=3, 
                      max_df=0.5, 
                      ngram_range=(1,2)
                     )

使用Count Vectorizer来fit训练集和测试集(半监督学习)

In [128]:
ctv.fit(list(xtrain) + list(xvalid)) 
xtrain_ctv = ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)



xdata_ctv = ctv.transform(xdata)
ydata_ctv = ctv.transform(ydata)

# 正确

利用提取的word counts特征来fit一个简单的Logistic Regression .

In [18]:
clf = LogisticRegression(C=1.0,solver='lbfgs',multi_class='multinomial') 
clf.fit(xtrain_ctv, ytrain) 
predictions = clf.predict_proba(xvalid_ctv) 

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 1.600 


比之前者,效果略差。。。 

接下来,尝试一个非常简单的模型- 朴素贝叶斯。 

利用提取的TFIDF特征来拟合Naive Bayes 

In [19]:
clf = MultinomialNB() 
clf.fit(xtrain_tfv, ytrain) 
predictions = clf.predict_proba(xvalid_tfv) 


print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.904 


效果更差。。。

朴素贝叶斯模型的表现不怎么样! 

但基于词汇计数的逻辑回归的效果仍然很棒! 

当我们在基于词汇计数的基础上使用朴素贝叶斯模型时会发生什么? 

利用提取的word counts特征来fitNaive Bayes 

In [20]:
clf = MultinomialNB() 
clf.fit(xtrain_ctv, ytrain) 
predictions = clf.predict_proba(xvalid_ctv) 
print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 6.870 


效果不好。。。

改用SVM

由于SVM需要花费大量时间,因此在应用SVM之前,我们将使用奇异值分解(Singular Value Decomposition )来减少TF-IDF中的特征数量。

同时,在使用SVM之前,我们还需要将数据标准化(Standardize Data ) 

使用SVD进行降维,components设为120,对于SVM来说,SVD的components的合适调整区间一般为120~200 。 

In [21]:
svd = decomposition.TruncatedSVD(n_components=120) 
svd.fit(xtrain_tfv) 


TruncatedSVD(algorithm='randomized', n_components=120, n_iter=5,
       random_state=None, tol=0.0)

In [22]:
xtrain_svd = svd.transform(xtrain_tfv) 
xvalid_svd = svd.transform(xvalid_tfv)

对从SVD获得的数据进行缩放:

In [23]:
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

调用下SVM模型：

In [24]:
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.635 


看起来，SVM在这些数据上表现还行！

在采用更高级的算法前，让我们再试试Kaggle上应用最流行的算法：xgboost！

基于tf-idf特征，使用xgboost

In [25]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.614 


效果不错，比SVM还牛呢！

## 基于word counts特征，使用xgboost：


In [48]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


logloss: 0.590 


In [133]:
type(predictions)

numpy.ndarray

In [146]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xdata_ctv.tocsc(), y)





XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=10, objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=0.8, verbosity=1)

In [143]:
# predictions

array([[ 0.4464027 ,  0.53692979,  0.01666747],
       [ 0.10165917,  0.48607007,  0.41227075],
       [ 0.0044926 ,  0.86899853,  0.12650888],
       ..., 
       [ 0.24134403,  0.75696921,  0.00168679],
       [ 0.14721394,  0.84507471,  0.00771137],
       [ 0.09660997,  0.87886864,  0.02452142]], dtype=float32)

In [147]:
predictions = clf.predict_proba(ydata_ctv.tocsc())
preds=np.argmax(predictions,axis=1)
test_pred=pd.DataFrame(preds)
test_pred.columns=["label"]
print(test_pred.shape)


(7356, 1)


In [148]:
test_data

,id,title,content,X,X_split
0,00005a3efe934a19adc0b69b05faeae7,九江办好人民满意教育,近3年来，九江市紧紧围绕“人本教育、公平教育、优质教育、幸福教育”的目标，努力办好人民满意教...,九江办好人民满意教育近3年来，九江市紧紧围绕“人本教育、公平教育、优质教育、幸福教育”的目标...,九江 办好 人民满意 教育 近 3 年来 ， 九江市 紧紧围绕 “ 人 本 教育 、 公平 ...
1,0009dc82c41341d794837271d4dbff5f,中央第三生态环境保护督察组转办我市第三十一批信访件办理情况,中央第三生态环境保护督察组转办我市的第三十一批信访件共计1件，截至12月11日，已全部办结，...,中央第三生态环境保护督察组转办我市第三十一批信访件办理情况中央第三生态环境保护督察组转办我市...,中央 第三 生态 环境保护 督察组 转办 我市 第三十 一批 信访件 办理 情况 中央 第三...
2,000f3763b6074588817c9ab90a22a814,大雨天车被淹，保险公司该不该赔？,核心提示：近日，连续的降雨天气造成中心城区部分路段积水严重，一些市民驾车出行涉水时造成车辆被...,大雨天车被淹，保险公司该不该赔？核心提示：近日，连续的降雨天气造成中心城区部分路段积水严重，...,大雨天 车 被淹 ， 保险公司 该不该 赔 ？ 核心 提示 ： 近日 ， 连续 的 降雨 天...
3,00117934dbe240068c95b6d04b08eea7,英特尔新cpu微架构ocean cove曝光,intel在主流平台上确认，今年将推出基于14nm的whiskeylake，明年开始大规模出...,英特尔新cpu微架构oceancove曝光intel在主流平台上确认，今年将推出基于14nm...,英特尔 新 cpu 微 架构 oceancove 曝光 intel 在 主流 平台 上 确认...
4,0014cef5ccfa43b3a6b20162a03763fa,公安部侦破一批重大网络赌博案件 德州约局平台成重灾区,2018年4月，公安部指挥河南、北京、广西等地公安机关联合行动，成功侦破北京联众公司棋牌事业...,公安部侦破一批重大网络赌博案件德州约局平台成重灾区2018年4月，公安部指挥河南、北京、广西...,公安部 侦破 一批 重大 网络 赌博 案件 德州 约局 平台 成 重灾区 2018 年 4 ...
5,0015fb20dbd54d249a50559789f43c52,广西龙胜千层天梯闹春耕,“五一”小长假期间，广西龙胜各族自治县龙脊古壮寨举办传统农事活动——“开耕节”，当地壮族同胞...,广西龙胜千层天梯闹春耕“五一”小长假期间，广西龙胜各族自治县龙脊古壮寨举办传统农事活动——“...,广西 龙胜 千层 天梯 闹 春耕 “ 五一 ” 小 长假 期间 ， 广西 龙胜各族自治县 龙...
6,002a0b77fded4ae1abaa8ed9f2ad0824,高唐小伙:挺身而出救交警,“这都是俺应该做的，交警作为人民群众的保护者，当看到他们受到伤害时，俺相信每个人都会勇于站出...,高唐小伙:挺身而出救交警“这都是俺应该做的，交警作为人民群众的保护者，当看到他们受到伤害时，...,高唐 小伙 : 挺身而出 救 交警 “ 这 都 是 俺 应该 做 的 ， 交警 作为 人民 ...
7,002c023ddb7343fdb8b3652ace2ded15,东营共享单车远走青岛，中间到底发生了什么?,本来应该在东营的共享电单车，位置却显示在青岛平度。咋回事?哈罗单车工作人员通过后台监控系统追...,东营共享单车远走青岛，中间到底发生了什么?本来应该在东营的共享电单车，位置却显示在青岛平度。...,东营 共享 单车 远走 青岛 ， 中间 到底 发生 了 什么 ? 本来 应该 在 东营 的 ...
8,002daa98e1b240679212a1b7a0402996,已有19人被捕!你的老婆可能还在做…赶紧看!,在在朋友圈看到微商已经不是什么稀罕的事打开朋友圈随便一刷很多都是卖保健品、膏药、榨汁机之类的...,已有19人被捕!你的老婆可能还在做…赶紧看!在在朋友圈看到微商已经不是什么稀罕的事打开朋友圈...,已有 19 人 被捕 ! 你 的 老婆 可能 还 在 做 … 赶紧 看 ! 在 在 朋友圈 ...
9,00305089bee5422ca2423176c448e91a,这两名“飙车党”太猖狂，竟当着重庆民警面玩漂移,公路上的摩托“飙车党”一直是警方重点打击对象。昨（25）日，重庆渝北区分局交巡警支队两路大队...,这两名“飙车党”太猖狂，竟当着重庆民警面玩漂移公路上的摩托“飙车党”一直是警方重点打击对象。...,这 两名 “ 飙车 党 ” 太 猖狂 ， 竟 当着 重庆 民警 面玩 漂移 公路 上 的 摩...


In [ ]:
test_pred["id"]=list(test_data["id"])
test_pred[["id","label"]].to_csv('1111111.csv',index=None)

In [59]:
clf.fit(xtrain_ctv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_ctv.tocsc())

print("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


<6538x166728 sparse matrix of type '<class 'numpy.int64'>'
	with 1972150 stored elements in Compressed Sparse Column format>

In [55]:
xTrain_tfidf = tfv.fit_transform(xTrain)
xTest_tfidf = tfv.transform(xTest)
yTrain=dataDropNa["label"]
 
#训练逻辑回归模型
clf = LogisticRegression(C=4, dual=True)
clf.fit(xTrain_tfidf, yTrain)
 

numpy.ndarray

In [ ]:
clt.fit()
preds=clf.predict(xTest_tfidf)
test_pred=pd.DataFrame(preds)
test_pred.columns=["label"]
test_pred["id"]=list(testData["id"])
test_pred[["id","label"]].to_csv('sub_svm_baseline.csv',index=None)
 

基于tf-idf的svd特征，使用xgboost

In [27]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.659 


再对经过数据标准化(Scaling)的tf-idf-svd特征使用xgboost

In [28]:
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.646 


XGBoost的效果似乎挺棒的！ 但我觉得还可以进一步优化，因为我还没有做过任何
超参数优化。 我很懒，所以我会告诉你该怎么做，你可以自己做！）。 这将在下
一节中讨论：

**网格搜索（Grid Search)**

网格搜索是一种超参数优化的技巧。 如果知道这个技巧，你可以通过获取最优的参数组合来产生良好的文本分类效果。

在本节中，我将讨论使用基于逻辑回归模型的网格搜索。

在开始网格搜索之前，我们需要创建一个评分函数，这可以通过scikit-learn
的make_scorer函数完成的。

In [29]:
mll_scorer = metrics.make_scorer(multiclass_logloss, 
                       greater_is_better=False, needs_proba=True)

接下来，我们需要一个pipeline。 为了演示，我将使用由SVD（进行特征缩放）和逻辑回归模型组成的pipeline。

In [30]:
#SVD初始化
svd = TruncatedSVD()

# Standard Scaler初始化
scl = preprocessing.StandardScaler()

# 再一次使用Logistic Regression
lr_model = LogisticRegression()

# 创建pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

接下来我们需要一个参数网格（A Grid of Parameters）：

In [31]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

因此，对于SVD，我们评估120和180个分量（Components），对于逻辑回归，我们评估三个不同的学习率C值，其中惩罚函数为l1和l2。 现在，我们可以开始对这些参数进行网格搜索咯。

网格搜索模型（Grid Search Model）初始化：

In [32]:
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

fit网格搜索模型:

In [ ]:
model.fit(xtrain_tfv, ytrain)  #为了减少计算量，这里我们仅使用xtrain


In [ ]:
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
nb_model = MultinomialNB()

创建pipeline :

In [ ]:
clf = pipeline.Pipeline([('nb', nb_model)])

搜索参数设置:

In [ ]:
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

网格搜索模型（Grid Search Model）初始化:

In [ ]:
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

fit网格搜索模型:

In [ ]:
model.fit(xtrain_tfv, ytrain)  # 为了减少计算量，这里我们仅使用xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

在NLP问题中，我们习惯于检索词向量（Word Vectors）。 Word向量提供了很多关
于数据的见解。 接下来，让我们来深入研究一下。

 ## 基于word2vec的词嵌入

在不深入细节的情况下，笔者将解释如何创建语句向量（Sentence Vectors），以
及如何基于它们在其上创建机器学习模型。鄙人是GloVe向量，word2vec和
fasttext的粉丝（但平时还是用word2vec较多）。在这篇文章中，笔者使用的文本
分类模型是基于Word2vec词向量模型（100维）。

训练word2vec词向量:

In [31]:
import gensim
model = gensim.models.Word2Vec(data, size=100)

X是经分词后的文本构成的list，也就是tokens的列表的列表.

    注意，Word2Vec还有3个值得关注的参数，iter是模型训练时迭代的次数，假
    如参与训练的文本量较少，就需要把这个参数调大一些；sg是模型训练算法的
    类别，1 代表 skip-gram，;0代表 CBOW;window控制窗口，它指当前词和预测
    词之间的最大距离，如果设得较小，那么模型学习到的是词汇间的功能性特征
    （词性相异），如果设置得较大，会学习到词汇之间的相似性特征（词性相
    同）的大小，假如语料够多，笔者一般会设置得大一些，8~10。


In [32]:
embeddings_index = dict(zip(model.wv.index2word, model.wv.syn0))

print('Found %s word vectors.' % len(embeddings_index))

C:\Users\dongy\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  if __name__ == '__main__':


Found 1 word vectors.


该函数会将语句转化为一个标准化的向量（Normalized Vector）

In [ ]:
def sent2vec(s):
#     words = str(s).lower()
    words = word_tokenize(words)
#     words = [w for w in words if not w in stop_words]
#     words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

对训练集和验证集使用上述函数，进行文本向量化处理

In [ ]:
xtrain_w2v  = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_w2v  = [sent2vec(x) for x in tqdm(xvalid)]

xtrain_w2v  = np.array(xtrain_w2v)
xvalid_w2v  = np.array(xvalid_w2v)

让我们看看xgboost在Glove词向量特征的表现如何：

基于word2vec特征使用XGB文本分类器：

In [ ]:
clf = xgb.XGBClassifier(nthread=10, silent=False)
clf.fit(xtrain_glove, ytrain)
predictions = clf.predict_proba(xvalid_w2v)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

In [ ]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1, silent=False)
clf.fit(xtrain_w2v, ytrain)
predictions = clf.predict_proba(xvalid_w2v)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

我们可以看到，简单的对参数进行微调，就提高基于word2vec词向量特征的
xgboost得分！ 相信我，你还可以从中继续“压榨”出更优秀的表现！


 深度学习（Deep Learning）

这是一个深度学习大行其道的时代！ 文本分类问题在它的指引下得到了突飞猛进
的发展！ 在这里，我们将在word2vec功能上训练LSTM和简单的全连接网络（Dense
Network）。
让我们先从全连接网络开始：

在使用神经网络前，对数据进行缩放：

In [ ]:
scl = preprocessing.StandardScaler()
xtrain_w2v_scl = scl.fit_transform(xtrain_w2v)
xvalid_w2v_scl = scl.transform(xvalid_w2v)